In [1]:
#Using melbourne dataset
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

#Load the data
data_path = "/Users/amishra/Documents/Machine Learning, AI/Courses/Kaggle-Machine-Learning-Courses/Data/melb_data.csv"
data = pd.read_csv(data_path)

#Select target
y = data.Price

#To keep things simple, we will use only numerical data
melb_predictors = data.drop(['Price'], axis = 1)
X = melb_predictors.select_dtypes(exclude =['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8,
                                                     test_size = 0.2, random_state =0)

In [2]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    return(mean_absolute_error(y_valid, predictions))

### Approach 1 : Remove the columns which has missing values

In [3]:
#Get names of columns iwth missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

#Drop columns with missing values in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis = 1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis = 1)

print("MAE from approach 1 Dropping entire columns with missing values:")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from approach 1 Dropping entire columns with missing values:
183550.22137772635


### Approach 2 : Imputation

In [9]:
from sklearn.impute import SimpleImputer

#Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

#Imputation removed columns. Put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 Imputation :")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 Imputation :
179816.89508731329


### Approach 3 : Imputation with extension

In [10]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid[col].isnull()
    

my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.fit_transform(X_valid_plus))

imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 Imputation with Extension :")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 Imputation with Extension :
179986.2708570026


In [13]:
print(X_train.shape)

(10864, 12)


In [15]:
missing_val_count_by_column = X_train.isnull().sum()
missing_val_count_by_column

Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                49
Landsize            0
BuildingArea     5156
YearBuilt        4307
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64

In [16]:
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


In [17]:
type(missing_val_count_by_column)

pandas.core.series.Series